In [1]:
import warnings
warnings.filterwarnings("ignore")
from copy import deepcopy
from datetime import datetime
from pathlib import Path

import numpy as np
import pandas as pd
import sys

sys.path.append("/code")

from tqdm import tqdm
import torch
import time
# device = torch.device('cpu')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
# import gym
# import recogym

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim

torch.backends.cudnn.benchmark = torch.cuda.is_available()
if torch.cuda.is_available():
    torch.set_float32_matmul_precision("high")  # TF32 = big speedup on Ada


from sklearn.utils import check_random_state

# implementing OPE of the IPWLearner using synthetic bandit data
from sklearn.linear_model import LogisticRegression

import matplotlib.pyplot as plt

from scipy.special import softmax
import optuna
# from memory_profiler import profile


from estimators import (
    DirectMethod as DM
)

from simulation_utils import (
    eval_policy,
    generate_dataset,
    create_simulation_data_from_pi,
    get_train_data,
    get_opl_results_dict,
    CustomCFDataset,
    calc_reward,
    get_weights_info
)

from models import (    
    LinearCFModel,
    NeighborhoodModel,
    BPRModel, 
    RegressionModel
)

from training_utils import (
    train,
    validation_loop, 
    cv_score_model
 )

from custom_losses import (
    SNDRPolicyLoss,
    IPWPolicyLoss, 
    KLPolicyLoss
    )

random_state=12345
random_ = check_random_state(random_state)

pd.options.display.float_format = '{:,.8f}'.format

Using device: cuda
Using device: cuda
Using device: cuda


In [2]:
def get_trial_results(
    our_x, 
    our_a, 
    emb_x, 
    emb_a, 
    original_x, 
    original_a, 
    dataset, 
    val_data, 
    original_policy_prob, 
    neighberhoodmodel, 
    regression_model, 
    dm
):
    t0 = time.time()
    policy = np.expand_dims(softmax(our_x @ our_a.T, axis=1), -1)
    policy_reward = calc_reward(dataset, policy)
    eval_metrics = eval_policy(neighberhoodmodel, val_data, original_policy_prob, policy)
    action_diff_to_real = np.sqrt(np.mean((emb_a - our_a) ** 2))
    action_delta = np.sqrt(np.mean((original_a - our_a) ** 2))
    context_diff_to_real = np.sqrt(np.mean((emb_x - our_x) ** 2))
    context_delta = np.sqrt(np.mean((original_x - our_x) ** 2))

    row = np.concatenate([
        np.atleast_1d(policy_reward),
        np.atleast_1d(eval_metrics),
        np.atleast_1d(action_diff_to_real),
        np.atleast_1d(action_delta),
        np.atleast_1d(context_diff_to_real),
        np.atleast_1d(context_delta)
    ])
    reg_dm = dm.estimate_policy_value(policy[val_data['x_idx']], regression_model.predict(val_data['x']))
    reg_results = np.array([reg_dm])
    conv_results = np.array([row])
    print(f"Evaluation total results time: {time.time() - t0} seconds")
    return get_opl_results_dict(reg_results, conv_results)

## `trainer_trial` Function

This function runs policy learning experiments using offline bandit data and evaluates various estimators.

### Parameters
- **num_runs** (int): Number of experimental runs per training size
- **num_neighbors** (int): Number of neighbors to consider in the neighborhood model
- **num_rounds_list** (list): List of training set sizes to evaluate
- **dataset** (dict): Contains dataset information including embeddings, action probabilities, and reward probabilities
- **batch_size** (int): Batch size for training the policy model
- **num_epochs** (int): Number of training epochs for each experiment
- **lr** (float, default=0.001): Learning rate for the optimizer

### Process Flow
1. Initializes result structures and retrieval models
2. For each training size in `num_rounds_list`:
   - Creates a uniform logging policy and simulates data
   - Generates training data for offline learning
   - Fits regression and neighborhood models for reward estimation
   - Initializes and trains a counterfactual policy model
   - Evaluates policy performance using various estimators
   - Collects metrics on policy reward and embedding quality

### Returns
- **DataFrame**: Results table with rows indexed by training size and columns for various metrics:
  - `policy_rewards`: True expected reward of the learned policy
  - Various estimator errors (`ipw`, `reg_dm`, `conv_dm`, `conv_dr`, `conv_sndr`)
  - Variance metrics for each estimator
  - Embedding quality metrics comparing learned representations to ground truth

### Implementation Notes
- Uses uniform random logging policy for collecting offline data
- Employs Self-Normalized Doubly Robust (SNDR) policy learning
- Measures embedding quality via RMSE to original/ground truth embeddings

In [3]:
def trainer_trial(
    num_runs,
    num_neighbors,
    train_sizes,
    dataset,
    batch_size,
    val_size=2000,
    n_trials=10,    
    prev_best_params=None
):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    torch.backends.cudnn.benchmark = torch.cuda.is_available()
    if torch.cuda.is_available():
        torch.set_float32_matmul_precision("high")

    dm = DM()
    results = {}

    our_x, our_a = dataset["our_x"], dataset["our_a"]
    emb_x, emb_a = dataset["emb_x"], dataset["emb_a"]

    original_x, original_a = dataset["original_x"], dataset["original_a"]
    n_users, n_actions, emb_dim = dataset["n_users"], dataset["n_actions"], dataset["emb_dim"]

    all_user_indices = np.arange(n_users, dtype=np.int64)

    def T(x):
        return torch.as_tensor(x, device=device, dtype=torch.float32)

    def _mean_dict(dicts):
        """
        Robust mean over a list of dicts with numeric/scalar/1D-array values.
        Returns a single dict with elementwise means.
        """
        if not dicts:
            return {}
        keys = dicts[0].keys()
        out = {}
        for k in keys:
            vals = [d[k] for d in dicts if k in d]
            # try to convert each to np.array and average
            arrs = [np.asarray(v) for v in vals]
            # broadcast to same shape if scalars/1D
            stacked = np.stack(arrs, axis=0)
            out[k] = np.mean(stacked, axis=0)
        return out

    # ===== unpack dataset (keep originals safe) =====
    our_x_orig, our_a_orig = our_x, our_a
    emb_x, emb_a = emb_x, emb_a
    original_x, original_a = original_x, original_a
    n_users, n_actions, emb_dim = n_users, n_actions, emb_dim
    all_user_indices = np.arange(n_users, dtype=np.int64)

    dm = DM()
    results = {}
    best_hyperparams_by_size = {}
    last_best_params = prev_best_params if prev_best_params is not None else None

    # ===== baseline (sample size = 0) using get_trial_results =====
    pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
    original_policy_prob = np.expand_dims(pi_0, -1)

    simulation_data = create_simulation_data_from_pi(
        dataset, pi_0, val_size, random_state=0
    )

    # use same data for train/val just to generate the baseline row
    train_data = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)
    val_data   = get_train_data(n_actions, val_size, simulation_data, np.arange(val_size), our_x_orig)
    t0 = time.time()
    regression_model = RegressionModel(
        n_actions=n_actions, action_context=our_x_orig,
        base_model=LogisticRegression(random_state=12345)
    )

    regression_model.fit(train_data['x'], train_data['a'], train_data['r'])
    print(f"Baseline regression model fit time: {time.time() - t0} seconds")

    t0 = time.time()
    neighberhoodmodel = NeighborhoodModel(
        train_data['x_idx'], train_data['a'],
        our_a_orig, our_x_orig, train_data['r'],
        num_neighbors=num_neighbors
    )
    print(f"Baseline neighborhood model fit time: {time.time() - t0} seconds")
    
    # baseline row produced via get_trial_results
    results[0] = get_trial_results(
        our_x_orig, our_a_orig, emb_x, emb_a, original_x, original_a,
        dataset, val_data, original_policy_prob,
        neighberhoodmodel, regression_model, dm
    )

    # ===== main loop over training sizes =====
    for train_size in train_sizes:

        # we’ll collect per-run trial dicts generated by get_trial_results
        trial_dicts_this_size = []
        best_hyperparams_by_size[train_size] = {}

        # --- prepare a resampling for Optuna’s objective (shared loaders built per-run inside objective) ---
        # We’ll do Optuna per-run (fresh resample + search), then final fit with best params, then get_trial_results.

        for run in range(num_runs):

            # --- resample for this run ---
            pi_0 = softmax(our_x_orig @ our_a_orig.T, axis=1)
            original_policy_prob = np.expand_dims(pi_0, -1)

            simulation_data = create_simulation_data_from_pi(
                dataset, pi_0, train_size + val_size,
                random_state=(run + 1) * (train_size + 17)
            )
                        
            idx_train = np.arange(train_size)
            train_data = get_train_data(n_actions, train_size, simulation_data, idx_train, our_x_orig)
            val_idx   = np.arange(val_size) + train_size
            val_data  = get_train_data(n_actions, val_size, simulation_data, val_idx, our_x_orig)

            num_workers = 4 if torch.cuda.is_available() else 0

            cf_dataset = CustomCFDataset(
                train_data['x_idx'], train_data['a'], train_data['r'], original_policy_prob
            )

            # val_loader = DataLoader(
            #     val_dataset, batch_size=val_size, shuffle=False,
            #     pin_memory=torch.cuda.is_available(),
            #     num_workers=num_workers, persistent_workers=bool(num_workers)
            # )


            # --- Optuna objective bound to this run's data ---
            def objective(trial):
                print()
                print(f"Trial {trial.number} started")
                lr = trial.suggest_float("lr", 1e-4, 1e-1, log=True)
                epochs = trial.suggest_int("num_epochs", 1, 10)
                trial_batch_size = trial.suggest_categorical("batch_size", [64, 128, 256, 512])
                trial_num_neighbors = trial.suggest_int("num_neighbors", 3, 15)
                lr_decay = trial.suggest_float("lr_decay", 0.8, 1.0)

                trial_neigh_model = NeighborhoodModel(
                    train_data['x_idx'], train_data['a'],
                    our_a_orig, our_x_orig, train_data['r'],
                    num_neighbors=trial_num_neighbors
                )

                trial_scores_all = torch.as_tensor(
                    trial_neigh_model.predict(all_user_indices),
                    device=device, dtype=torch.float32
                )

                trial_model = LinearCFModel(
                    n_users, n_actions, emb_dim,
                    initial_user_embeddings=T(our_x_orig),
                    initial_actions_embeddings=T(our_a_orig)
                ).to(device)

                assert (not torch.cuda.is_available()) or next(trial_model.parameters()).is_cuda

                final_train_loader = DataLoader(
                    cf_dataset, batch_size=trial_batch_size, shuffle=True,
                    pin_memory=torch.cuda.is_available(),
                    num_workers=num_workers, persistent_workers=bool(num_workers)
                )

                current_lr = lr
                for epoch in range(epochs):
                    if epoch > 0:
                        current_lr *= lr_decay
                        
                    train(
                        trial_model, final_train_loader, trial_scores_all,
                        criterion=KLPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                    )

                trial_x, trial_a = trial_model.get_params()
                trial_x = trial_x.detach().cpu().numpy()
                trial_a = trial_a.detach().cpu().numpy()

                pi_i = softmax(trial_x @ trial_a.T, axis=1)
                train_actions = train_data['a']
                train_users = train_data['x_idx']

                print("Train wi info: {}".format(get_weights_info(pi_i[train_users, train_actions], original_policy_prob[train_users, train_actions])))
                print(f"actual reward: {calc_reward(dataset, np.expand_dims(pi_i, -1))}")

                # print(get_weights_info(pi_i, original_policy_prob))
                # validation reward for selection
                return cv_score_model(val_data, trial_scores_all, pi_i)


            # --- run Optuna for this run ---
            study = optuna.create_study(direction="maximize")
            
            if last_best_params is not None:
                study.enqueue_trial(last_best_params)

            study.optimize(objective, n_trials=n_trials, show_progress_bar=True)

            best_params = study.best_params
            last_best_params = best_params  # optional warm-start to next run
            best_hyperparams_by_size[train_size][run] = {
                "params": best_params,
                "reward": study.best_value
            }


            # --- final training with best params on this run’s data ---
            regression_model = RegressionModel(
                n_actions=n_actions, action_context=our_x_orig,
                base_model=LogisticRegression(random_state=12345)
            )
            regression_model.fit(
                train_data['x'], train_data['a'], train_data['r'],
                original_policy_prob[train_data['x_idx'], train_data['a']].squeeze()
            )

            neighberhoodmodel = NeighborhoodModel(
                train_data['x_idx'], train_data['a'],
                our_a_orig, our_x_orig, train_data['r'],
                num_neighbors=best_params['num_neighbors']
            )
            scores_all = torch.as_tensor(
                neighberhoodmodel.predict(all_user_indices),
                device=device, dtype=torch.float32
            )

            model = LinearCFModel(
                n_users, n_actions, emb_dim,
                initial_user_embeddings=T(our_x_orig),
                initial_actions_embeddings=T(our_a_orig)
            ).to(device)
            assert (not torch.cuda.is_available()) or next(model.parameters()).is_cuda

            train_loader = DataLoader(
                cf_dataset, batch_size=batch_size, shuffle=True,
                pin_memory=torch.cuda.is_available(),
                num_workers=num_workers, persistent_workers=bool(num_workers)
            )

            current_lr = best_params['lr']
            for epoch in range(best_params['num_epochs']):
                if epoch > 0:
                    current_lr *= best_params['lr_decay']
                train(
                    model, train_loader, scores_all,
                    criterion=KLPolicyLoss(), num_epochs=1, lr=current_lr, device=str(device)
                )

            # learned embeddings (do NOT overwrite originals)
            learned_x_t, learned_a_t = model.get_params()
            learned_x = learned_x_t.detach().cpu().numpy()
            learned_a = learned_a_t.detach().cpu().numpy()

            # --- produce the per-run result via get_trial_results ---
            trial_res = get_trial_results(
                learned_x, learned_a,          # learned (policy) embeddings
                emb_x, emb_a,                  # ground-truth embedding refs
                original_x, original_a,        # original clean refs
                dataset,
                val_data,                      # use this run's val split
                original_policy_prob,
                neighberhoodmodel,
                regression_model,
                dm
            )

            trial_dicts_this_size.append(trial_res)

            # memory hygiene
            torch.cuda.empty_cache()

        # === aggregate per-run results (mean) and store under this train_size ===
        results[train_size] = _mean_dict(trial_dicts_this_size)

    return pd.DataFrame.from_dict(results, orient='index'), best_hyperparams_by_size

## Learning

We will run several simulations on a generated dataset, the dataset is generated like this:
$$ \text{We have users U and actions A } u_i \sim N(0, I_{emb_dim}) \ a_i \sim N(0, I_{emb_dim})$$
$$ p_{ij} = 1 / (5 + e^{-(u_i.T a_j)}) $$
$$r_{ij} \sim Bin(p_{ij})$$

We have a policy $\pi$
and it's ground truth reward is calculated by
$$R_{gt} = \sum_{i}{\sum_{j}{\pi_{ij} * p_{ij}}} $$

Our parameters for the dataset will be
$$EmbDim = 5$$
$$NumActions= 150$$
$$NumUsers = 150$$
$$NeighborhoodSize = 6$$

to learn a new policy from $\pi$ we will sample from:
$$\pi_{start} = (1-\epsilon)*\pi + \epsilon * \pi_{random}$$

In [4]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params)

Random Item CTR: 0.07066414727263938
Optimal greedy CTR: 0.09999926940951757
Second Best greedy CTR: 0.0980913477695915
Optimal Stochastic CTR: 0.09995326955796031
second Best Stochastic CTR: 0.08595012935428775
Our Initial CTR: 0.08610747363354625


In [5]:
num_runs = 1
batch_size = 200
num_neighbors = 6
n_trials_for_optuna = 20
# num_rounds_list = [500, 1000, 2000, 10000, 20000]
# num_rounds_list = [500, 1000, 2000]
num_rounds_list = [15000]


# Manually define your best parameters
best_params_to_use = {
    "lr": 0.096,  # Learning rate
    "num_epochs": 5,  # Number of training epochs
    "batch_size": 64,  # Batch size for training
    "num_neighbors": 8,  # Number of neighbors for neighborhood model
    "lr_decay": 0.85  # Learning rate decay factor
}

### 1

$$emb = 0.7 * gt + 0.3 * noise$$
$$lr = 0.005$$
$$n_{epochs} = 1$$
$$BatchSize=50$$

In [6]:
print("Value of num_rounds_list:", num_rounds_list)

# Run the optimization
df4, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# # Print best hyperparameters for each training size
# print("\n=== BEST HYPERPARAMETERS BY TRAINING SIZE ===")
# for train_size, params in best_hyperparams_by_size.items():
#     print(f"\nTraining Size: {train_size}")
#     # print(f"Best Reward: {params['reward']:.6f}")
#     print("Parameters:")
#     for param_name, value in params['params'].items():
#         print(f"  {param_name}: {value}")
# print("===========================\n")

# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Value of num_rounds_list: [15000]
Simulation time for 10000 samples: 0.026812314987182617 seconds
Baseline regression model fit time: 0.08083295822143555 seconds
Baseline neighborhood model fit time: 29.70861315727234 seconds
Num samples is 10000
{'gini': np.float64(0.48357763099887546), 'ess': np.float64(3891.817348806044), 'max_wi': np.float64(45.00658812185537), 'min_wi': np.float64(0.009939141556591315)}
Eval time: 0.14032912254333496 seconds


[I 2025-11-02 21:16:12,715] A new study created in memory with name: no-name-092da759-c53a-4fbb-ba31-23320c36d88f


Evaluation total results time: 0.48679256439208984 seconds
Simulation time for 25000 samples: 0.05918002128601074 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


...

Train wi info: {'gini': np.float64(0.9518297338774782), 'ess': np.float64(43.46950616018178), 'max_wi': np.float64(2242.0391936677065), 'min_wi': np.float64(4.324676004668218e-06)}
actual reward: [0.08943429]
{'gini': np.float64(0.9424889824845832), 'ess': np.float64(35.03038279707777), 'max_wi': np.float64(1572.6067255187463), 'min_wi': np.float64(4.324676004668218e-06)}
Estimated reward: 0.104737
Cross-validated error: 0.005783
Final score CI (reward +- 2*error): [0.093172, 0.116303]
Standard error: 0.011879
Final t_dist CI (reward +- t_0.975*se_hat): [0.081453, 0.128022]
[I 2025-11-02 21:32:21,210] Trial 18 finished with value: 0.0931720892674161 and parameters: {'lr': 0.00308032468803497, 'num_epochs': 7, 'batch_size': 128, 'num_neighbors': 15, 'lr_decay': 0.9230119525572414}. Best is trial 18 with value: 0.0931720892674161.

Trial 19 started


Best trial: 18. Best value: 0.0931721: 100%|██████████| 20/20 [17:01<00:00, 51.06s/it]

Train wi info: {'gini': np.float64(0.776764947013987), 'ess': np.float64(2561.3302782660808), 'max_wi': np.float64(21.24674187623402), 'min_wi': np.float64(0.0030063969800626865)}
actual reward: [0.08716433]
{'gini': np.float64(0.765499305841617), 'ess': np.float64(1825.685461694621), 'max_wi': np.float64(23.615959658617033), 'min_wi': np.float64(0.0032950065787286445)}
Estimated reward: 0.087751
Cross-validated error: 0.002122
Final score CI (reward +- 2*error): [0.083507, 0.091996]
Standard error: 0.006920
Final t_dist CI (reward +- t_0.975*se_hat): [0.074187, 0.101315]
[I 2025-11-02 21:33:13,919] Trial 19 finished with value: 0.08350693662902288 and parameters: {'lr': 0.0008971223503204559, 'num_epochs': 9, 'batch_size': 128, 'num_neighbors': 15, 'lr_decay': 0.9312294812405533}. Best is trial 18 with value: 0.0931720892674161.


Num samples is 10000
{'gini': np.float64(0.9360595847771348), 'ess': np.float64(307.40500003324627), 'max_wi': np.float64(185.78904343050002), 'min_wi': np.float64(2.0316160421014675e-05)}
Eval time: 0.11384701728820801 seconds
Evaluation total results time: 0.4126303195953369 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08580000,0.08587958,0.09146488,0.08702020,0.08702020,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08986485,0.08938016,0.08942195,0.09146836,0.09230034,0.09230175,0.84992686,0.24763566,0.92641604,0.15059657


### Policy with delta function

In [7]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=10000)

Random Item CTR: 0.07083863592474163
Optimal greedy CTR: 0.09999916436977967
Second Best greedy CTR: 0.08797326118616329
Optimal Stochastic CTR: 0.0999493542444427
second Best Stochastic CTR: 0.0854530317781557
Our Initial CTR: 0.08557719469284641


In [8]:
# Run the optimization
df5, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df5[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.026820898056030273 seconds
Baseline regression model fit time: 0.08495235443115234 seconds
Baseline neighborhood model fit time: 30.432530641555786 seconds
Num samples is 10000
{'gini': np.float64(0.4624242051067587), 'ess': np.float64(4432.5227373596235), 'max_wi': np.float64(37.468396402411294), 'min_wi': np.float64(0.02084098835003422)}
Eval time: 0.14079689979553223 seconds


[I 2025-11-02 21:34:30,448] A new study created in memory with name: no-name-6a0fbb1a-5c60-4c6d-bfdd-51e0b4be8e5f


Evaluation total results time: 0.4810309410095215 seconds
Simulation time for 25000 samples: 0.061788320541381836 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


...

Train wi info: {'gini': np.float64(0.9880517063091835), 'ess': np.float64(75.47153963179706), 'max_wi': np.float64(413.98828557969875), 'min_wi': np.float64(3.341185367083889e-09)}
actual reward: [0.0861028]
{'gini': np.float64(0.9880013893001773), 'ess': np.float64(61.25499798451042), 'max_wi': np.float64(287.7370713369324), 'min_wi': np.float64(3.341185367083889e-09)}
Estimated reward: 0.078673
Cross-validated error: 0.007799
Final score CI (reward +- 2*error): [0.063076, 0.094270]
Standard error: 0.013554
Final t_dist CI (reward +- t_0.975*se_hat): [0.052104, 0.105242]
[I 2025-11-02 21:50:32,472] Trial 18 finished with value: 0.06307571039476496 and parameters: {'lr': 0.006830366072451752, 'num_epochs': 5, 'batch_size': 128, 'num_neighbors': 9, 'lr_decay': 0.9408773811114716}. Best is trial 14 with value: 0.07705365655814808.

Trial 19 started


Best trial: 14. Best value: 0.0770537: 100%|██████████| 20/20 [16:52<00:00, 50.63s/it]

Train wi info: {'gini': np.float64(0.9991137711877944), 'ess': np.float64(9.941047089300197), 'max_wi': np.float64(1922.7868040482715), 'min_wi': np.float64(4.373249415554434e-15)}
actual reward: [0.0820328]
{'gini': np.float64(0.9993146384114582), 'ess': np.float64(5.4865715573957266), 'max_wi': np.float64(7035.41729969867), 'min_wi': np.float64(5.768955090271751e-15)}
Estimated reward: 0.011054
Cross-validated error: 0.002692
Final score CI (reward +- 2*error): [0.005671, 0.016437]
Standard error: 0.024755
Final t_dist CI (reward +- t_0.975*se_hat): [-0.037470, 0.059578]
[I 2025-11-02 21:51:23,098] Trial 19 finished with value: 0.0056705500254202994 and parameters: {'lr': 0.04922187681474791, 'num_epochs': 7, 'batch_size': 512, 'num_neighbors': 12, 'lr_decay': 0.863423725647143}. Best is trial 14 with value: 0.07705365655814808.


Num samples is 10000
{'gini': np.float64(0.9931155080650808), 'ess': np.float64(34.09260205722292), 'max_wi': np.float64(756.3882061438497), 'min_wi': np.float64(2.758324709017361e-09)}
Eval time: 0.1286313533782959 seconds
Evaluation total results time: 0.4351184368133545 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08557719,0.08660000,0.08650364,0.08909426,0.08686976,0.08686976,0.82618217,0.00000000,0.99950468,0.00000000
15000,0.08668432,0.06059182,0.08585125,0.10038258,0.07450582,0.07469615,0.96631088,0.37342811,1.10425372,0.21374903


In [9]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=20000)

Random Item CTR: 0.07042251854546815
Optimal greedy CTR: 0.09999934264692525
Second Best greedy CTR: 0.09938443255799592
Optimal Stochastic CTR: 0.09996075464321043
second Best Stochastic CTR: 0.08632684639469405
Our Initial CTR: 0.08647580588501355


In [10]:
# Run the optimization
df6, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df6[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.0241241455078125 seconds
Baseline regression model fit time: 0.08198142051696777 seconds
Baseline neighborhood model fit time: 25.81819438934326 seconds
Num samples is 10000
{'gini': np.float64(0.4768924046597257), 'ess': np.float64(4272.9908177404395), 'max_wi': np.float64(20.503708318829123), 'min_wi': np.float64(0.008806514039618792)}
Eval time: 0.1249399185180664 seconds


[I 2025-11-02 21:52:42,608] A new study created in memory with name: no-name-27b11f15-9667-4b35-9646-1a0f881b1380


Evaluation total results time: 0.42638134956359863 seconds
Simulation time for 25000 samples: 0.05681633949279785 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


...

Train wi info: {'gini': np.float64(0.08662460758596519), 'ess': np.float64(14631.636290668011), 'max_wi': np.float64(1.6440226130681312), 'min_wi': np.float64(0.6509547233844817)}
actual reward: [0.08666602]
{'gini': np.float64(0.0824681587850956), 'ess': np.float64(9778.51053340677), 'max_wi': np.float64(1.6440226130681312), 'min_wi': np.float64(0.6726316284171544)}
Estimated reward: 0.075997
Cross-validated error: 0.000795
Final score CI (reward +- 2*error): [0.074406, 0.077587]
Standard error: 0.002878
Final t_dist CI (reward +- t_0.975*se_hat): [0.070355, 0.081639]
[I 2025-11-02 22:08:29,355] Trial 18 finished with value: 0.0744060798066481 and parameters: {'lr': 0.00025589434366040563, 'num_epochs': 9, 'batch_size': 256, 'num_neighbors': 7, 'lr_decay': 0.8569639395793818}. Best is trial 9 with value: 0.07527034086424444.

Trial 19 started


Best trial: 9. Best value: 0.0752703: 100%|██████████| 20/20 [16:38<00:00, 49.95s/it]

Train wi info: {'gini': np.float64(0.9705187450561152), 'ess': np.float64(328.84510558226236), 'max_wi': np.float64(94.83859658763726), 'min_wi': np.float64(2.1708930197552057e-06)}
actual reward: [0.09179006]
{'gini': np.float64(0.9603075831133824), 'ess': np.float64(252.50512751695004), 'max_wi': np.float64(106.52646096458814), 'min_wi': np.float64(1.128372166231466e-06)}
Estimated reward: 0.069584
Cross-validated error: 0.004720
Final score CI (reward +- 2*error): [0.060144, 0.079023]
Standard error: 0.015913
Final t_dist CI (reward +- t_0.975*se_hat): [0.038392, 0.100775]
[I 2025-11-02 22:09:21,543] Trial 19 finished with value: 0.06014435308839608 and parameters: {'lr': 0.015414231825338109, 'num_epochs': 9, 'batch_size': 512, 'num_neighbors': 12, 'lr_decay': 0.8148879849976876}. Best is trial 9 with value: 0.07527034086424444.


Num samples is 10000
{'gini': np.float64(0.48388758204171756), 'ess': np.float64(3955.3919197282758), 'max_wi': np.float64(39.46599916552396), 'min_wi': np.float64(0.013699470010061249)}
Eval time: 0.13395237922668457 seconds
Evaluation total results time: 0.45945310592651367 seconds


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647581,0.08700000,0.08702125,0.09039638,0.08857681,0.08857681,0.88083979,0.00000000,0.74725465,0.00000000
15000,0.08655371,0.07801639,0.08690984,0.08939873,0.07697208,0.07696609,0.88327901,0.00874626,0.74708647,0.00417677


In [11]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=30000)

Random Item CTR: 0.07069350185865088
Optimal greedy CTR: 0.09999918303816259
Second Best greedy CTR: 0.09988806364453348
Optimal Stochastic CTR: 0.0999509448932121
second Best Stochastic CTR: 0.0863879153702632
Our Initial CTR: 0.08653966603258505


In [ ]:
# Run the optimization
df7, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df7[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Simulation time for 10000 samples: 0.031176328659057617 seconds
Baseline regression model fit time: 0.0773158073425293 seconds
Baseline neighborhood model fit time: 28.948025465011597 seconds
Num samples is 10000
{'gini': np.float64(0.48828470612959685), 'ess': np.float64(4030.997503021888), 'max_wi': np.float64(26.91439561207676), 'min_wi': np.float64(0.012952797711514308)}
Eval time: 0.13135600090026855 seconds


[I 2025-11-02 22:10:48,484] A new study created in memory with name: no-name-e6f8a2c3-8e38-453c-a7c0-bd241be2de62


Evaluation total results time: 0.46826839447021484 seconds
Simulation time for 25000 samples: 0.05841255187988281 seconds


  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.107449:   5%|▌         | 1/20 [00:45<14:33, 46.00s/it]

Train wi info: {'gini': np.float64(0.988980790854492), 'ess': np.float64(180.27680009282193), 'max_wi': np.float64(264.098491568967), 'min_wi': np.float64(0.0)}
actual reward: [0.08437358]
{'gini': np.float64(0.9946821726377266), 'ess': np.float64(40.70282750449641), 'max_wi': np.float64(931.3444355746129), 'min_wi': np.float64(0.0)}
Estimated reward: 0.132892
Cross-validated error: 0.012721
Final score CI (reward +- 2*error): [0.107449, 0.158334]
Standard error: 0.038127
Final t_dist CI (reward +- t_0.975*se_hat): [0.058155, 0.207628]
[I 2025-11-02 22:11:34,481] Trial 0 finished with value: 0.10744872990421701 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.10744872990421701.

Trial 1 started


Best trial: 0. Best value: 0.107449:  10%|█         | 2/20 [01:33<14:00, 46.69s/it]

Train wi info: {'gini': np.float64(0.8365404243901994), 'ess': np.float64(1128.874030946434), 'max_wi': np.float64(114.02907033449958), 'min_wi': np.float64(0.0009032113650532207)}
actual reward: [0.09018413]
{'gini': np.float64(0.8201184048298633), 'ess': np.float64(295.24002557159395), 'max_wi': np.float64(484.69982333574035), 'min_wi': np.float64(0.0009032113650532207)}
Estimated reward: 0.081052
Cross-validated error: 0.003223
Final score CI (reward +- 2*error): [0.074606, 0.087497]
Standard error: 0.012018
Final t_dist CI (reward +- t_0.975*se_hat): [0.057495, 0.104609]
[I 2025-11-02 22:12:21,661] Trial 1 finished with value: 0.07460615361326069 and parameters: {'lr': 0.010131288988147595, 'num_epochs': 1, 'batch_size': 64, 'num_neighbors': 4, 'lr_decay': 0.8218244099936143}. Best is trial 0 with value: 0.10744872990421701.

Trial 2 started


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=40000)

Random Item CTR: 0.07053370144999074
Optimal greedy CTR: 0.09999936716169436
Second Best greedy CTR: 0.09676800930842865
Optimal Stochastic CTR: 0.09995563088920843
second Best Stochastic CTR: 0.08606322612964991
Our Initial CTR: 0.08622184481781218


In [ ]:
# Run the optimization
df8, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df8[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.48297294790935064), 'ess': np.float64(4206.88542516398), 'max_wi': np.float64(26.68727224829075), 'min_wi': np.float64(0.014262757298096463)}


[I 2025-10-30 10:03:03,724] A new study created in memory with name: no-name-31bed130-c819-466f-b871-4219b0b0ee45
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0689699:   5%|▌         | 1/20 [00:42<13:23, 42.27s/it]

Train wi info: {'gini': np.float64(0.5626145283674258), 'ess': np.float64(866.5206756290892), 'max_wi': np.float64(221.1270920127721), 'min_wi': np.float64(1.582570511292046e-08)}
actual reward: [0.07098819]
{'gini': np.float64(0.626748524747854), 'ess': np.float64(309.31416297110025), 'max_wi': np.float64(306.5124985861054), 'min_wi': np.float64(4.203406264184544e-09)}
Estimated reward: 0.075058
Cross-validated error: 0.003044
Final score CI (reward +- 2*error): [0.068970, 0.081146]
Standard error: 0.002512
Final t_dist CI (reward +- t_0.975*se_hat): [0.070135, 0.079982]
[I 2025-10-30 10:03:45,988] Trial 0 finished with value: 0.06896991747601121 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.06896991747601121.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.0007888421843980483), 'ess': np.float64(14999.969367799215), 'max_wi': np.float64(1.0071680557999327), 'min_wi': np.float64(0.9926170286304189)}
actual reward: [0.08622169]
{'gini': np.float64(0.000857575285339545), 'ess': np.float64(9999.976165835476), 'max_wi': np.float64(1.0061927260081673), 'min_wi': np.float64(0.9924287481271912)}
Estimated reward: 0.078685
Cross-validated error: 0.000882
Final score CI (reward +- 2*error): [0.076921, 0.080449]
Standard error: 0.002837
Final t_dist CI (reward +- t_0.975*se_hat): [0.073124, 0.084245]
[I 2025-10-30 10:19:06,760] Trial 18 finished with value: 0.0769209633382369 and parameters: {'lr': 0.0002752642380955787, 'num_epochs': 6, 'batch_size': 256, 'num_neighbors': 15, 'lr_decay': 0.8732271743681641}. Best is trial 14 with value: 0.07794662480323412.

Trial 19 started


Best trial: 14. Best value: 0.0779466: 100%|██████████| 20/20 [16:50<00:00, 50.55s/it]

Train wi info: {'gini': np.float64(0.0034579803629473343), 'ess': np.float64(14999.40919634976), 'max_wi': np.float64(1.0320677621018406), 'min_wi': np.float64(0.9626869805921305)}
actual reward: [0.0862182]
{'gini': np.float64(0.003892809550713829), 'ess': np.float64(9999.503779316083), 'max_wi': np.float64(1.0294673562624606), 'min_wi': np.float64(0.967206852483284)}
Estimated reward: 0.078547
Cross-validated error: 0.000974
Final score CI (reward +- 2*error): [0.076600, 0.080494]
Standard error: 0.002874
Final t_dist CI (reward +- t_0.975*se_hat): [0.072913, 0.084182]
[I 2025-10-30 10:19:54,628] Trial 19 finished with value: 0.07660004628349852 and parameters: {'lr': 0.0008804230446990676, 'num_epochs': 9, 'batch_size': 256, 'num_neighbors': 11, 'lr_decay': 0.9411507255309078}. Best is trial 14 with value: 0.07794662480323412.


Num samples is 10000
{'gini': np.float64(0.4668149495753226), 'ess': np.float64(4672.823503627641), 'max_wi': np.float64(16.962707883902738), 'min_wi': np.float64(0.014015010024919842)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08622184,0.08920000,0.08927464,0.09322973,0.08955775,0.08955775,0.92210476,0.00000000,0.83772226,0.00000000
15000,0.08622244,0.07799825,0.08640752,0.08976593,0.07853542,0.07853573,0.92212387,0.00159145,0.83767829,0.00168795


In [ ]:
dataset_params = dict(
                    n_actions= 500,
                    n_users = 500,
                    emb_dim = 16,
                    # sigma = 0.1,
                    eps = 0.6, # this is the epsilon for the noise in the ground truth policy representation
                    ctr = 0.1
                    )

train_dataset = generate_dataset(dataset_params, seed=50000)

Random Item CTR: 0.0705882181025533
Optimal greedy CTR: 0.09999934164533562
Second Best greedy CTR: 0.09924496289352924
Optimal Stochastic CTR: 0.09995498601895662
second Best Stochastic CTR: 0.08629000824986369
Our Initial CTR: 0.08647501952799874


In [ ]:
# Run the optimization
df9, best_hyperparams_by_size = trainer_trial(num_runs, num_neighbors, num_rounds_list, train_dataset, batch_size, val_size=10000, n_trials=n_trials_for_optuna, prev_best_params=best_params_to_use)

# Show the performance metrics
df9[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

Num samples is 10000
{'gini': np.float64(0.48655661699878366), 'ess': np.float64(4239.612735801623), 'max_wi': np.float64(18.4166650696614), 'min_wi': np.float64(0.019148344088431003)}


[I 2025-10-30 10:21:07,302] A new study created in memory with name: no-name-24f4bb67-00af-4afd-b9a6-22d92411903a
  0%|          | 0/20 [00:00<?, ?it/s]


Trial 0 started


Best trial: 0. Best value: 0.0649131:   5%|▌         | 1/20 [00:40<12:48, 40.46s/it]

Train wi info: {'gini': np.float64(0.5758156274969405), 'ess': np.float64(582.4190650954737), 'max_wi': np.float64(278.28867653625656), 'min_wi': np.float64(6.081468907580241e-05)}
actual reward: [0.07089183]
{'gini': np.float64(0.6483447842288079), 'ess': np.float64(114.6669352290496), 'max_wi': np.float64(581.304013527663), 'min_wi': np.float64(0.00011427346244316298)}
Estimated reward: 0.070481
Cross-validated error: 0.002784
Final score CI (reward +- 2*error): [0.064913, 0.076050]
Standard error: 0.002412
Final t_dist CI (reward +- t_0.975*se_hat): [0.065754, 0.075209]
[I 2025-10-30 10:21:47,756] Trial 0 finished with value: 0.06491312135708051 and parameters: {'lr': 0.096, 'num_epochs': 5, 'batch_size': 64, 'num_neighbors': 8, 'lr_decay': 0.85}. Best is trial 0 with value: 0.06491312135708051.

Trial 1 started


...

Train wi info: {'gini': np.float64(0.04482226270354083), 'ess': np.float64(14882.65724769548), 'max_wi': np.float64(1.7095539190138052), 'min_wi': np.float64(0.2002279705282591)}
actual reward: [0.08620516]
{'gini': np.float64(0.05325322448194675), 'ess': np.float64(9894.692264109874), 'max_wi': np.float64(1.5562116188919466), 'min_wi': np.float64(0.15915356651274173)}
Estimated reward: 0.079254
Cross-validated error: 0.000804
Final score CI (reward +- 2*error): [0.077646, 0.080862]
Standard error: 0.002939
Final t_dist CI (reward +- t_0.975*se_hat): [0.073493, 0.085015]
[I 2025-10-30 10:36:21,459] Trial 18 finished with value: 0.07764581574889343 and parameters: {'lr': 0.009204464019853708, 'num_epochs': 5, 'batch_size': 128, 'num_neighbors': 10, 'lr_decay': 0.8370472583835912}. Best is trial 2 with value: 0.0791962370252024.

Trial 19 started


Best trial: 2. Best value: 0.0791962: 100%|██████████| 20/20 [16:03<00:00, 48.17s/it]

Train wi info: {'gini': np.float64(0.01518064812998182), 'ess': np.float64(14988.575244758265), 'max_wi': np.float64(1.1354345201967349), 'min_wi': np.float64(0.803377001140173)}
actual reward: [0.08644032]
{'gini': np.float64(0.016122229364670868), 'ess': np.float64(9991.468862125128), 'max_wi': np.float64(1.14984807310634), 'min_wi': np.float64(0.7961880906819326)}
Estimated reward: 0.078175
Cross-validated error: 0.000860
Final score CI (reward +- 2*error): [0.076455, 0.079895]
Standard error: 0.002870
Final t_dist CI (reward +- t_0.975*se_hat): [0.072549, 0.083801]
[I 2025-10-30 10:37:10,745] Trial 19 finished with value: 0.07645546832418647 and parameters: {'lr': 0.0035825808692318537, 'num_epochs': 4, 'batch_size': 256, 'num_neighbors': 13, 'lr_decay': 0.8673327927964085}. Best is trial 2 with value: 0.0791962370252024.


Num samples is 10000
{'gini': np.float64(0.48036454227576), 'ess': np.float64(4271.377590547567), 'max_wi': np.float64(24.576691778849106), 'min_wi': np.float64(0.0124287350975967)}


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08647502,0.08750000,0.08750948,0.09359878,0.08782022,0.08782022,0.80232812,0.00000000,0.84032376,0.00000000
15000,0.08646534,0.07892782,0.08790025,0.09306271,0.07930589,0.07930759,0.80160775,0.00632052,0.84015372,0.00612846


In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08660000,0.08653168,0.09008066,0.08644724,0.08644724,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610802,0.07770118,0.08627892,0.08844592,0.07717319,0.07717316,0.75692672,0.00057714,0.87628001,0.00071563


### Poicy Via argmax(r_hat - error_hat) through cross validation

In [ ]:
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]

,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08660000,0.08653168,0.09008066,0.08644724,0.08644724,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610802,0.07770118,0.08627892,0.08844592,0.07717319,0.07717316,0.75692672,0.00057714,0.87628001,0.00071563


### Policy Via using actual policy value

In [ ]:
# Show the performance metrics
df4[['policy_rewards', 'ipw', 'reg_dm', 'conv_dm', 'conv_dr', 'conv_sndr', 'action_diff_to_real', 'action_delta', 'context_diff_to_real', 'context_delta']]


,policy_rewards,ipw,reg_dm,conv_dm,conv_dr,conv_sndr,action_diff_to_real,action_delta,context_diff_to_real,context_delta
0,0.08610747,0.08660000,0.08653168,0.09008066,0.08644724,0.08644724,0.75692870,0.00000000,0.87627132,0.00000000
15000,0.08610802,0.07770118,0.08627892,0.08844592,0.07717319,0.07717316,0.75692672,0.00057714,0.87628001,0.00071563
